<a href="https://colab.research.google.com/github/Manya123-max/Deep-Learning-Algorithms-/blob/main/DeepLab6_Encoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Input, Flatten, Dense, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers # Import regularizers from Keras

In [ ]:
# Data load, Data preprocessing, spliting - Train and Testing
from tensorflow.keras.datasets import mnist
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Define input shape
input_shape = (28, 28, 1) # Example shape for grayscale 28x28 images (e.g., MNIST)

In [ ]:
# Preprocess the data
def preprocess(array):
    array = array.astype("float32") / 255.0
    array = np.reshape(array, (len(array), 28, 28, 1))
    return array

# Load and preprocess Medical MNIST data
from tensorflow.keras.datasets import mnist  # Replace this with Medical MNIST loading code
(x_train, _), (x_test, _) = mnist.load_data()
x_train = preprocess(x_train)
x_test = preprocess(x_test)


In [ ]:
# Encoder
def regularized_encoder(input_layer):
    x = Conv2D(32, (3, 3), activation='relu', padding='same', activity_regularizer=regularizers.l1(10e-5))(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)  # Bottleneck layer
    return x

In [ ]:
# Decoder
def decoder(encoded_input):
    x = Dense(7 * 7 * 128, activation='relu')(encoded_input)
    x = Reshape((7, 7, 128))(x)
    x = Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)
    return x

In [ ]:
# Build the Autoencoder model
input_layer = Input(shape=(28, 28, 1))
encoded = regularized_encoder(input_layer)
decoded = decoder(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the model
autoencoder.fit(x_train, x_train, epochs=50, batch_size=128, shuffle=True, validation_data=(x_test, x_test))

# Visualize the model
autoencoder.summary()

Epoch 1/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - loss: 0.9240 - val_loss: 0.0937
Epoch 2/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 0.0159 - val_loss: 0.0208
Epoch 3/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 0.0129 - val_loss: 0.0150
Epoch 4/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0110 - val_loss: 0.0346
Epoch 5/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 0.0098 - val_loss: 0.0182
Epoch 6/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0089 - val_loss: 0.0311
Epoch 7/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 0.0084 - val_loss: 0.0214
Epoch 8/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0089 - val_loss: 0.0533
Epoch 9/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 0.0084 - val_loss: 0.0188
Epoch 10/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0075 - val_loss: 0.0117
Epoch 11/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0075 - val_loss: 0.0191
Epoch 12/50
469/469 ━━━━━━━━━━━━━━━━

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 28, 28, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 28, 28, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 14, 14, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 14, 14, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 7, 7, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 7, 7, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         401,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6272)                │         407,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 7, 7, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 7, 7, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 14, 14, 64)          │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 14, 14, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 28, 28, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 28, 28, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_3 (Conv2DTranspose) │ (None, 28, 28, 1)           │             289 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 3,429,445 (13.08 MB)

 Trainable params: 1,142,849 (4.36 MB)

 Non-trainable params: 896 (3.50 KB)

 Optimizer params: 2,285,700 (8.72 MB)

In [ ]:
# Preprocess the data
def preprocess(array):
    array = array.astype("float32") / 255.0
    array = np.reshape(array, (len(array), 28, 28, 1))
    return array

In [ ]:
# Add noise to the images
def add_noise(array, noise_factor=0.4):
    noisy_array = array + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=array.shape)
    return np.clip(noisy_array, 0.0, 1.0)

# Load and preprocess Medical MNIST data
from tensorflow.keras.datasets import mnist  # Replace with Medical MNIST loading code
(x_train, _), (x_test, _) = mnist.load_data()
x_train = preprocess(x_train)
x_test = preprocess(x_test)

# Add noise
noisy_x_train = add_noise(x_train)
noisy_x_test = add_noise(x_test)

In [ ]:
# Encoder
def denoised_encoder(input_layer):
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)  # Bottleneck layer
    return x

In [ ]:
# Decoder
def decoder(encoded_input):
    x = Dense(7 * 7 * 128, activation='relu')(encoded_input)
    x = Reshape((7, 7, 128))(x)
    x = Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same', strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)
    return x

In [ ]:
# Build the Denoised Autoencoder model
input_layer = Input(shape=(28, 28, 1))
encoded = denoised_encoder(input_layer)
decoded = decoder(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the model
autoencoder.fit(noisy_x_train, x_train, epochs=50, batch_size=128, shuffle=True, validation_data=(noisy_x_test, x_test))

# Visualize the model
autoencoder.summary()

Epoch 1/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - loss: 0.0546 - val_loss: 0.0152
Epoch 2/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - loss: 0.0098 - val_loss: 0.0097
Epoch 3/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 0.0085 - val_loss: 0.0113
Epoch 4/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 0.0079 - val_loss: 0.0087
Epoch 5/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0075 - val_loss: 0.0080
Epoch 6/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 0.0072 - val_loss: 0.0075
Epoch 7/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 0.0070 - val_loss: 0.0075
Epoch 8/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 0.0068 - val_loss: 0.0077
Epoch 9/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 0.0067 - val_loss: 0.0073
Epoch 10/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - loss: 0.0065 - val_loss: 0.0075
Epoch 11/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 0.0064 - val_loss: 0.0078
Epoch 12/50
469/469 ━━━━━━━━━━━

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 28, 28, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 14, 14, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 14, 14, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 7, 7, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 7, 7, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │         401,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 6272)                │         407,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_1 (Reshape)                  │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_4 (Conv2DTranspose) │ (None, 7, 7, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 7, 7, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_5 (Conv2DTranspose) │ (None, 14, 14, 64)          │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 14, 14, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_6 (Conv2DTranspose) │ (None, 28, 28, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 28, 28, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_7 (Conv2DTranspose) │ (None, 28, 28, 1)           │             289 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 3,429,445 (13.08 MB)

 Trainable params: 1,142,849 (4.36 MB)

 Non-trainable params: 896 (3.50 KB)

 Optimizer params: 2,285,700 (8.72 MB)